In [1]:
# import pandas
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# exchange codes http://finabase.blogspot.com/2014/09/interantional-stock-exchange-codes-for.html

path = './Compustat data.csv.nosync.csv'
df = pd.read_csv(path, sep=',')

market_caps_path = './market_caps.csv'
market_caps = pd.read_csv(market_caps_path, sep=',')


/Users/vini2001/miniforge3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,10,12,16,26,30,945,946,948,949,950,955,965,969,970,975) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
print(len(df))
# drop all df rows where exchg is [0, 1, 2, 19]
df = df[df['exchg'] != 0]
df = df[df['exchg'] != 1]
df = df[df['exchg'] != 2]
df = df[df['exchg'] != 13]
df = df[df['exchg'] != 19]

# drop rows without market value
df = df[df['mkvalt'].notna()]
print(len(df))

# df2021 = df[df['fyear'] == 2021]

# df2021 = df2021.drop_duplicates(subset=['gvkey'], keep='first')
# print(f'{len(df2021)} rows in total')

# map_exchg_to_count = df2021.groupby('exchg')['gvkey'].count()
# map_exchg_to_count

593074
129505


In [3]:
def market_caps_for_year(year):
    dfyear = df[df['fyear'] == year]
    # return df with mkvalt and tic
    dfyear = dfyear[['tic', 'mkvalt']]
    #drop nans
    dfyear = dfyear.dropna()
    return dfyear

In [4]:
# df_2020 = df[df['fyear'] == 2020.0]


# umns(df, min_count, file_tag):
#     vals = df_columns_2020[df_columns_2020['FileTag'] == file_tag]
#     vals = vals[vals['Count'] > 8000]['Column']
#     return vals.values.tolist()

# vals_bal_ann_indl = select_columns(df_columns_2020, 8000, 'BAL_ANN_INDL')

In [5]:

path_items_descriptions = './items_descriptions.csv'
df_items_descriptions = pd.read_csv(path_items_descriptions, sep=',')
def get_description(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['ItemName'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['ItemDesc']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def get_file_tag(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['FileTag'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['FileTag']
    return vals.values[0] if len(vals) > 0 else 'Not found'

get_file_tag('at')

'BAL_ANN_INDL'

In [92]:
columns = df.columns.tolist()
print(columns)

df.head()


tickers = df['tic']
# get set of unique tickers
tickers = set(tickers)
print(f'{len(tickers)} tickers')

#export tickers to csv
path = './tickers.csv'
tickers = pd.DataFrame(tickers)
tickers.to_csv(path, index=False)

df['fyear']

aapl_data = df[df['tic'] == 'AAPL']
# create csv file
path = '/Users/vini2001/Desktop/UFMG/AAPL.csv'
aapl_data.to_csv(path, index=False)

['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'act', 'adpac', 'aedi', 'afudcc', 'afudci', 'aldo', 'am', 'amc', 'amdc', 'amgw', 'ano', 'ao', 'aocidergl', 'aociother', 'aocipen', 'aocisecgl', 'aodo', 'aol2', 'aoloch', 'aox', 'ap', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqc', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'at', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'bcltbl', 'bcnlr', 'bcrbl', 'bct', 'bctbl', 'bctr', 'billexce', 'bkvlps', 'bltb

In [7]:
len(df)

129505

In [8]:

def most_frequent_columns(df):
    map_count = {}
    for c in columns:
        map_count[c] = df[c].describe()['count']

    # sort by count
    map_count = {k: v for k, v in sorted(map_count.items(), key=lambda item: item[1], reverse=True)}

    columns_data = []
    for k, v in map_count.items():
        columns_data.append([k, v, get_description(k), get_file_tag(k)])

    df_columns = pd.DataFrame(columns_data, columns=['Column', 'Count', 'Description', 'FileTag'])
    return df_columns

df_columns = most_frequent_columns(df)

path = './columns.csv'
df_columns.to_csv(path, index=False)
df_columns.head()

,Column,Count,Description,FileTag
0,gvkey,129505.0,Global Company Key - Key Selection,Multiple Files
1,datadate,129505.0,Data Date,Multiple Files
2,fyear,129505.0,Data Year - Fiscal,DES_ANN
3,indfmt,129505.0,Not found,Not found
4,consol,129505.0,Level of Consolidation,CCM_FILEDATE


In [9]:
rows_2020 = df[df['fyear'] == 2020.0]
rows_2020 = rows_2020.reset_index(drop=True)
tickers_2020 = rows_2020['tic']
tickers_2020 = set(tickers_2020)
print(f'{len(tickers_2020)} tickers in 2020')

df_columns_2020 = most_frequent_columns(rows_2020)
path = './columns_2020.csv'
df_columns_2020.to_csv(path, index=False)
df_columns_2020

4549 tickers in 2020


,Column,Count,Description,FileTag
0,gvkey,5153.0,Global Company Key - Key Selection,Multiple Files
1,datadate,5153.0,Data Date,Multiple Files
2,fyear,5153.0,Data Year - Fiscal,DES_ANN
3,indfmt,5153.0,Not found,Not found
4,consol,5153.0,Level of Consolidation,CCM_FILEDATE
...,...,...,...,...
976,xoptd,0.0,Implied Option EPS Diluted,IS_ANN_INDL
977,xopteps,0.0,Implied Option EPS Basic,IS_ANN_INDL
978,add3,0.0,Address Line 3,COMPANY
979,add4,0.0,Address Line 4,COMPANY


In [10]:
# select columns with count > min_count
def select_columns(df, min_count, file_tag):
    vals = df[df_columns_2020['FileTag'] == file_tag]
    vals = vals[vals['Count'] > min_count]['Column']
    return vals.values.tolist()

vals_bal_ann_indl = select_columns(df_columns_2020, 5000, 'BAL_ANN_INDL')
vals_is_ann_indl = select_columns(df_columns_2020, 5000, 'IS_ANN_INDL')

print(len(vals_bal_ann_indl))
print(len(vals_is_ann_indl))

# select all rows in rows_2020 with columns of vals_bal_ann_indl and vals_is_ann_indl different than NaN
rows_2020_bal_and_is = rows_2020[['tic', 'fyear'] + vals_bal_ann_indl + vals_is_ann_indl + ['mkvalt']]
rows_2020_bal_and_is = rows_2020_bal_and_is.dropna()
rows_2020_bal_and_is = rows_2020_bal_and_is.reset_index(drop=True)
print(len(rows_2020_bal_and_is))

# create csv file
path = './rows_2020_bal_ann_indl.csv'
rows_2020_bal_and_is.to_csv(path, index=False)

tickers = rows_2020_bal_and_is['tic']

18
9
4611


In [11]:
data = rows_2020_bal_and_is

# compustat already has market cap value, we don't need to get from yahoo finances, that can be unreliable
# data = rows_2020_bal_and_is

len(data)

4611

In [12]:
data.columns

Index(['tic', 'fyear', 'at', 'icapt', 'lse', 'ao', 'pstkr', 'tstkp', 'ceq',
       'pstkn', 'dltt', 'tstkc', 'dlc', 'gdwl', 'intan', 'ch', 'ivst', 'cstk',
       'intano', 'caps', 'dvc', 'dvt', 'revt', 'do', 'dvp', 'ib', 'xido',
       'cstke', 'ibmii', 'mkvalt', 'market_cap_yahoo_finances'],
      dtype='object')

In [13]:
#mlp regressor 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.kernel_ridge import KernelRidge



import numpy as np

# split in x and y
x = data.drop(columns=['tic', 'fyear', 'mkvalt', 'market_cap_yahoo_finances'])
y = data['mkvalt']

# normalize x and y
scaler = StandardScaler()
x = scaler.fit_transform(x)
# y = scaler.fit_transform(y.values.reshape(-1, 1))   

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# MLP Regressor
mlp_regressor = MLPRegressor(
    learning_rate='adaptive',
    max_iter=1000,
    hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100),
    activation='relu',
    solver='adam',
)

mlp_regressor.fit(x, y)
score = mlp_regressor.score(x, y)
print(f'MLP Regressor Score: {score}')


MLP Regressor Score: 0.6969354593090663


In [14]:


print(f'Analysing {len(data)} tickers')
mapT = {}

market_caps_2021 = market_caps_for_year(market_caps, 2021)
market_caps_2020 = market_caps_for_year(market_caps, 2020)

# loop through data
for i in range(len(data)):
    row = data.iloc[i]
    ticker = row['tic']
    x_i = x[i]
    # print(x_i)
    y_i = row['mkvalt']
    # mlp_regressor.predict([x_i])
    p = mlp_regressor.predict([x_i])
    diff = p[0]/y_i - 1
    mapT[ticker] = diff

# sort mapT by biggest diff
mapT = {k: v for k, v in sorted(mapT.items(), key=lambda item: item[1], reverse=True)}

# #get top 30
# top30 = list(mapT.keys())[:30]

# count = 0
# sumAll = 0
# for tic in top30:
#     vals_2021 = market_caps_2021[market_caps_2021['tic'] == tic]['mkvalt'].values
#     # we also exclude some tickers here that are presenting a huge change in market cap - they don't have historical prices for 2020 for some reason, doesn't seem like a good idea to use them
#     if len(vals_2021) == 0 or tic in ['NE', 'EVTL', 'VAL', 'CHK', 'GPOR', 'FYBR', 'CBL']:
#         continue
#     m2021 = vals_2021[0] 
#     m2020 = market_caps_2020[market_caps_2020['tic'] == tic]['mkvalt'].values[0] 
#     change = m2021/m2020-1
#     exchg = df[df['tic'] == tic]['exchg'].values[0]
#     print(f'{tic}: {mapT[tic]} => {change} ({exchg})')
#     sumAll += change
#     count += 1

# print(f'Average change: {sumAll/count}')

Analysing 4611 tickers


In [15]:
possible_invalid = []

# loop mapT
for tic, diff in mapT.items():
    if diff < 0:
        diff = 1/(diff+1)
    if diff > 10:
        # print(f'{tic}: {diff}')
        possible_invalid.append(tic)

print(f'Possible invalid count: {len(possible_invalid)}')

# for tic in possible_invalid:
#     exchg = df[df['tic'] == tic]['exchg'].values[0]
#     print(f'{tic} ({exchg})')

Possible invalid count: 1401


In [16]:
for tic in possible_invalid:
    dt = data[data['tic'] == tic]
    dt = dt[dt['fyear'] == 2020.0]
    print(f'{tic}: {dt["mkvalt"].values[0]} x  {dt["market_cap_yahoo_finances"].values[0]}')

DTLA.P: 0.0144 x  nan
PMX.: 0.8455 x  nan
CMO.H: 1.0531 x  nan
EVTL: 1.0553 x  2256737569.9367657
MCA.H: 1.3651 x  nan
NE: 6.2771 x  nan
CEO.: 1.5427 x  nan
VDO.H: 1.5885 x  nan
LIM.H: 1.6011 x  nan
VAL: 10.0174 x  nan
GOE.H: 1.7132 x  nan
AB.: 1.8441 x  nan
NLTA.: 1.8789 x  nan
SVFD: 1.9766 x  38591224.71365457
BHR.1: 2.2873 x  nan
HPI.: 2.3362 x  nan
CHK: 14.8671 x  nan
GPOR: 6.6716 x  nan
MNK: 17.0056 x  nan
EFF.: 2.789 x  nan
GOK.: 2.9959 x  nan
FYBR: 9.379 x  nan
TWY.: 3.1622 x  nan
ARTL: 4.7923 x  nan
SBFM: 5.4388 x  nan
XOG: 3.8791 x  nan
SSY: 6.1746 x  8831770.243265394
TCCO: 6.3085 x  8864032.389068604
RUM.: 6.4112 x  nan
MAZ.H: 6.4485 x  nan
NTN: 6.6214 x  nan
PRO.: 6.6553 x  nan
MTSL: 6.8588 x  nan
BCTX: 6.8951 x  64710058.3423621
TUEM: 7.5793 x  nan
BRN: 7.057 x  11932800.624610718
AIMD: 7.1554 x  nan
NUTX: 7.1321 x  nan
DO: 23.7177 x  nan
BTCS: 7.9823 x  nan
KTRA: 7.8808 x  62125174.40181868
CBL: 8.1577 x  nan
RCON: 8.2114 x  49687029.06520528
CEAD: 8.2548 x  nan
MOSY: 8.6

In [17]:
bigDiffs = []
notFoundYf = []
correct = 0
for tic in tickers:
    tic_df = df[df['tic'] == tic][['mkvalt', 'fyear']]
    tic_df_2020 = tic_df[tic_df['fyear'] == 2020.0]
    compustat = tic_df_2020['mkvalt'].values[0]*1e6
    mc_2020 = market_caps_for_year(market_caps, 2020)
    vals = mc_2020[mc_2020['tic'] == tic]['2020']
    if len(vals) == 0: 
        notFoundYf.append(tic)
        continue
    yf = vals.values[0]
    # if yf is NaN continue
    if np.isnan(yf):
        notFoundYf.append(tic)
        continue
    diff = yf/compustat if yf > compustat else compustat/yf
    if diff > 4:
        bigDiffs.append(tic)
        print(f'{tic}: {diff}')
    else: correct += 1

print(f'Correct: {correct}, total: {len(tickers)}, big diffs: {len(bigDiffs)}')


GTBP: 5.860809652182129
JOB: 6.458368710024339
FLUX: 4.64326981706118
RGLS: 21.64680982488208
FKWL: 4.653951630175968
LEVI: 5.596307121977445
ANY: 8.012324167769767
KTRA: 7.883105065706359
RZLT: 6.7858188107009365
GLG: 8.730711254179583
AIKI: 43.484278930285846
TRVN: 27.137655870445336
COCP: 16.604807940278743
AGRX: 18.55305735116061
SLNO: 15.035887851238446
TTOO: 56.040344628522256
RMBL: 7.22288176807887
KPRX: 91.09496106129687
VANI: 6.555476814545827
VIVE: 4.883739136326426
BCTX: 9.384933988247031
ESTE: 4.603664477843119
AVGR: 22.500146303564236
KMPH: 7.60273333655177
ATXS: 4.534351777502796
BTB: 8.386690661143872
PULM: 31.158583750554346
FCEL: 6.948829257801705
NBRV: 67.2703585359102
AVTX: 17.9699175177193
PXS: 7.733187608769476
ZDGE: 5.093051228178638
FWBI: 142.5101123595506
PIXY: 35.85422274250055
VRDN: 5.134078645790796
KALA: 55.58881361974531
XELA: 26.367837773089725
AVCT: 66.10319951399788
BRQS: 4.978677558204761
AIHS: 9.485642086850051
FAMI: 37.20197350875634
SLDB: 19.48830401

In [18]:
for t in notFoundYf:
    x = data[data['tic'] == t]
    y = x['mkvalt']
    x = x.drop(columns=['tic', 'fyear', 'mkvalt', 'market_cap_yahoo_finances'])

    # normalize x and y
    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    # mlp_regressor.predict
    # print(t)
    p = mlp_regressor.predict(x)[0]
    # print(p)
    c = data[data['tic'] == t]['mkvalt'].values[0]
    # print(c)
    # print(data[data['tic'] == t]['market_cap_yahoo_finances'].values[0] / 1e6)
    # print()

    if(p / c > 50): 
        if t == 'NE': print(f'{t}: {p / c}')
        print()

    # TODO: se nao achar o ticker aqui (https://stockanalysis.com/stocks/cspi/market-cap/), descarta ele

# data[data['tic'] == 'GTBP']





































NE: 1289.7357190249381












































































































































































































































































































































































































In [19]:
df[df['tic'] == 'CSPI'][df['fyear'] == 2020.0]['mkvalt'].values[0] 

<ipython-input-19-2418014d9f24>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['tic'] == 'CSPI'][df['fyear'] == 2020.0]['mkvalt'].values[0]


36.9019

In [110]:
df[df['tic'] == 'SSY'][['fyear', 'datadate', 'csho', 'prcc_f', 'mkvalt', 'tstkn', 'cshrt']]

,fyear,datadate,csho,prcc_f,mkvalt,tstkn,cshrt
205044,1998.0,19990331,4.978,1.5000,7.4670,0.278,NaN
205045,1999.0,20000331,4.976,1.3750,6.8420,0.000,NaN
205046,2000.0,20010331,4.976,1.4900,7.4142,0.000,NaN
205047,2001.0,20010630,4.976,2.3000,11.4448,0.000,NaN
205048,2002.0,20020630,4.998,3.1500,15.7437,0.000,NaN
205049,2003.0,20030630,5.028,2.4100,12.1175,0.000,NaN
205050,2004.0,20040630,7.072,5.5500,39.2496,0.000,NaN
205051,2005.0,20050630,7.198,7.8800,56.7202,0.000,NaN
205052,2006.0,20060630,7.315,9.9000,72.4185,0.000,NaN
205053,2007.0,20070630,7.510,6.3200,47.4632,0.000,NaN


In [108]:
dfevtl = df[df['tic'] == 'OBE']
dfevtl[['fyear', 'datadate', 'prcc_f', 'csho', 'ajex', 'mkvalt', 'exchg',]]


,fyear,datadate,prcc_f,csho,ajex,mkvalt,exchg
326533,1998.0,19981231,16.50,45.278,0.4286,747.0870,12.0
326534,1999.0,19991231,28.25,51.222,0.4286,1447.0215,12.0
326535,2000.0,20001231,37.40,51.818,0.4286,1937.9932,12.0
326536,2001.0,20011231,35.40,52.723,0.4286,1866.3942,12.0
326537,2002.0,20021231,41.00,53.733,0.4286,2203.0530,12.0
326538,2003.0,20031231,48.17,53.692,0.4286,2586.3436,12.0
326539,2004.0,20041231,79.25,53.869,0.4286,4269.1183,12.0
326540,2005.0,20051231,37.99,163.290,0.1429,6203.3871,12.0
326541,2006.0,20061231,35.57,237.126,0.1429,8434.5718,12.0
326542,2007.0,20071231,25.84,242.663,0.1429,6270.4119,12.0


In [85]:
count_big_diffs = 0
count_total = 0
for t in tickers:
    dft = df[df['tic'] == t]
    csho_2020 = dft[dft['fyear'] == 2020.0]['csho'].values[0]
    vals = dft[dft['fyear'] == 2021.0]['csho'].values
    if len(vals) == 0: continue
    csho_2021 = vals[0]

    ajex_2020 = dft[dft['fyear'] == 2020.0]['ajex'].values[0]
    ajex_2021 = dft[dft['fyear'] == 2021.0]['ajex'].values[0]

    if csho_2021/ajex_2021 < csho_2020/ajex_2020:
        count_big_diffs += 1
        print(f'{t}: {csho_2021/csho_2020}')
    count_total += 1


print(f'count_big_diffs: {count_big_diffs}, count_total: {count_total}, count_tickers: {len(tickers)}')

CECO: 0.9904147934515226
PRG: 0.8334096244998007
ABT: 0.9959643863304032
AMD: 0.9966969446738233
MATX: 0.949074074074074
HON: 0.9827784156142365
SWKS: 0.9981884057971016
AXP: 0.9453416149068323
AXP: 0.9453416149068323
AMGN: 0.9654158741137818
AXR: 0.7155537348081388
AXR: 0.7155537348081388
APA: 0.9190663395870531
APOG: 0.9217157968421872
AMAT: 0.975929978118162
TREC: 0.949945636854186
ARCB: 0.9794784937765874
ARW: 0.9094504496078457
ASH: 0.9344262295081968
REX: 0.9879839786381842
ADSK: 0.9936247723132968
ADP: 0.985578041404978
AVY: 0.9934456591021154
AZZ: 0.9832722638202962
BCPC: 0.9973434652333736
BALL: 0.979419144896302
BPOP: 0.9478538530933953
BPOP: 0.9478538530933953
BSET: 0.9817962385597909
BAX: 0.9934642464573245
AIT: 0.9949883750968741
BDX: 0.9808942234420741
BBY: 0.8848249027237355
VAL: 0.37584565271861686
RHI: 0.9784138321193693
RFP: 0.9504950495049505
BMY: 0.9696969696969697
CAL: 0.99128167307591
BC: 0.9879036918138042
CTS: 0.9969637822598135
CACI: 0.9386681544653886
CNI: 0.9